In [ ]:
import pandas as pd
import os

def process_and_save_data(input_folder, output_folder):
    # Pridobimo seznam vseh datotek v vhodni mapi, ki niso že obdelane
    files_to_process = [file for file in os.listdir(input_folder) if 'processed' not in file]

    for file_to_process in files_to_process:
        # Preberemo podatke iz CSV datoteke
        input_csv_path = os.path.join(input_folder, file_to_process)
        data = pd.read_csv(input_csv_path)

        # Pretvorimo časovne žige v stolpcu "last_update" v časovni tip podatka
        data['last_update'] = pd.to_datetime(data['last_update'], unit='ms')
        
        # Zaokrožimo časovne žige na urni interval
        data['datetime'] = data['last_update'].dt.floor('H')

        # Združimo podatke po postajališčih in urah
        aggregated_data = data.groupby(['name', 'datetime']).agg({
            'available_bikes': 'mean',
            'available_bike_stands': 'mean'
        }).reset_index()

        # Seznam vseh postajališč
        postajalisca = aggregated_data['name'].unique()

        for postajalisce in postajalisca:
            # Filtriramo podatke za trenutno postajališče
            filtered_data = aggregated_data[aggregated_data['name'] == postajalisce]
            
            # Ime datoteke za shranjevanje podatkov
            ime_datoteke = os.path.join(output_folder, f"{postajalisce.replace(' ', '_')}.csv")
            
            # Shranimo filtrirane podatke v CSV datoteko
            # z načinom 'w' za pisanje, da se podatki prepišejo
            filtered_data.to_csv(ime_datoteke, mode='w', index=False)

            print(f"Podatki za postajališče {postajalisce} so bili uspešno prepisani v datoteko {ime_datoteke}")

        # Preimenovanje datoteke, ki jo obdelamo
        processed_file_path_renamed = os.path.join(input_folder, f"processed_{file_to_process}")
        os.rename(input_csv_path, processed_file_path_renamed)

        print(f"Datoteka {file_to_process} je bila preimenovana v {processed_file_path_renamed}")

# Uporaba funkcije
input_folder = '../data/raw/fetch_mbajk.csv'
output_folder = '../data/processed'
process_and_save_data(input_folder, output_folder)
